In [1]:
%load_ext rich

# Pre-built modules

In [2]:
import promptimus as pm

In [3]:
provider = pm.llms.OpenAILike(
    model_name="qwen/qwen-2-7b-instruct:free", base_url="https://openrouter.ai/api/v1"
)

## Memory Module

In [4]:
assistant = pm.modules.MemoryModule(
    memory_size=3, system_prompt="You are an assistant."
).with_provider(provider)

In [5]:
await assistant.forward("Hi my name is Ailadin!")


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content='Hello Ailadin, nice to meet you! How can I assist you today?'
)

In [6]:
# remembering name from context
await assistant.forward("What is my name?")

Message(role=<MessageRole.ASSISTANT: 'assistant'>, content='Your name is Ailadin.')

In [7]:
await assistant.forward("What is your name?")


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content="As an artificial intelligence, I don't have a personal name, but you can call me Assistant. How can I assist you today?"
)

In [8]:
await assistant.forward("What is my the weather today?")


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content="I'm sorry for the confusion, but as an AI text-based model running on a computer, I do not have the capability to access real-time data, gather information or browse the internet. I suggest using a weather-app or visiting a relevant website to find your current weather conditions."
)

In [9]:
# memory maxlen exhausted
assistant.memory

Memory[deque([Message(role=<MessageRole.ASSISTANT: 'assistant'>, content="As an artificial intelligence, I don't have a personal name, but you can call me Assistant. How can I assist you today?"), Message(role=<MessageRole.USER: 'user'>, content='What is my the weather today?'), Message(role=<MessageRole.ASSISTANT: 'assistant'>, content="I'm sorry for the confusion, but as an AI text-based model running on a computer, I do not have the capability to access real-time data, gather information or browse the internet. I suggest using a weather-app or visiting a relevant website to find your current weather conditions.")], maxlen=3)]

In [10]:
# now it doesn't have memory in context
await assistant.forward("Can you recall my name?")


Message(
    role=<MessageRole.ASSISTANT: 'assistant'>,
    content='As an AI, I don\'t retain information between sessions and don\'t have the capability to remember personal data unless you specifically tell me something like "My name is [Your Name]". If you\'d like to remind me, just say, "My name is [Your Name]."'
)

In [11]:
# memory can be cleaned manually
assistant.memory.reset()
assistant.memory

Memory[deque([], maxlen=3)]

In [12]:
# or can be cleared on context enter and exit

with assistant.memory:
    await assistant.forward("Hi my name is Ailadin")
    await assistant.forward("What is my name")
    print("Inside context", assistant.memory)
print("----")
print("Outside context", assistant.memory)

Inside context Memory[deque([Message(role=<MessageRole.ASSISTANT: 'assistant'>, content="Hello Ailadin, it's nice to meet you! How can I assist you today?"), Message(role=<MessageRole.USER: 'user'>, content='What is my name'), Message(role=<MessageRole.ASSISTANT: 'assistant'>, content='Your name is Ailadin.')], maxlen=3)]
----
Outside context Memory[deque([], maxlen=3)]


In [13]:
assistant.save("assets/step_3_memory.toml")
!cat assets/step_3_memory.toml

prompt = """
You are an assistant.
"""



## Structural Output

In [14]:
from enum import StrEnum, auto

from pydantic import BaseModel, Field

In [15]:
class Operation(StrEnum):
    SUM = auto()
    SUB = auto()
    DIV = auto()
    MUL = auto()


class CalculatorSchema(BaseModel):
    reasoning: str
    a: float = Field(description="The left operand.")
    b: float = Field(description="The right operand.")
    op: Operation = Field(description="The operation to execute.")


module = pm.modules.StructuralOutput(CalculatorSchema).with_provider(provider)

In [17]:
await module.forward("I have 10 cows, I need twice the amount")


CalculatorSchema(
    reasoning='To acquire twice the amount of current cows (10 cows), you need to purchase an additional 10 cows.',
    a=10.0,
    b=2.0,
    op=<Operation.MUL: 'mul'>
)

In [22]:
await module.forward("I have 10 cows, i need half of them")


CalculatorSchema(
    reasoning='Calculating half of the number of cows to understand the quantity needed.',
    a=10.0,
    b=2.0,
    op=<Operation.DIV: 'div'>
)

In [19]:
await module.forward("I have 11.11 cows. and 11 left the herd")


CalculatorSchema(
    reasoning='To find out how many cows are left after 11 left the herd',
    a=11.11,
    b=11.0,
    op=<Operation.SUB: 'sub'>
)

In [20]:
await module.forward("I have 5 cows. and my friend have 7")

CalculatorSchema(reasoning='Calculating total number of cows', a=5.0, b=7.0, op=<Operation.SUM: 'sum'>)

In [21]:
module.save("assets/step_3_structural.toml")
!cat assets/step_3_structural.toml

retry_template = """
Your response does not conform to the required schema. Please correct your output by ensuring it matches the expected format and constraints. 

**Schema Validation Error:**  
`{error_message}`  
  
Please reattempt the response, ensuring strict adherence to the schema.    
"""


[predictor]
prompt = """
You are designed to generate structured JSON outputs adhering to a predefined schema without any embellishments or formatting.

Schema Description: {{
    "$defs": {{
        "Operation": {{
            "enum": [
                "sum",
                "sub",
                "div",
                "mul"
            ],
            "title": "Operation",
            "type": "string"
        }}
    }},
    "properties": {{
        "reasoning": {{
            "title": "Reasoning",
            "type": "string"
        }},
        "a": {{
            "description": "The left operand.",
            "title": "A",
            "type": "number"
        }},
        "b": {{
      